In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Colab Notebooks/Progetti Personali/foto_ale

/content/gdrive/MyDrive/Colab Notebooks/Progetti Personali/foto_ale


In [3]:
pip install -r requirements.txt

     |████████████████████████████████| 100.2MB 46kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=1fe186bafa27a55e1a532aca3d794fd5be2702f18b020e8416ed15c4e9ec3f04
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [4]:
import face_recognition
import os
import re
from PIL import Image
import pickle
from tqdm.notebook import tqdm
import numpy as np
import cv2
import gc

shape = 800

path_embedding = '/content/gdrive/MyDrive/Progetti Personali/foto_ale/baseline/embedding/embedding_list.pkl'
path_original = "/content/gdrive/MyDrive/Progetti Personali/foto_ale/Foto All"

path_save = '/content/gdrive/MyDrive/Progetti Personali/foto_ale/save/list_file/'
image_pattern = r".*\.(jpg|jpeg|png)"

In [5]:
original_photo_path = [os.path.join(path, f) for path, subdirs, files in os.walk(path_original)  for f in files if re.match(image_pattern, f, flags=re.I)]
print(len(original_photo_path))

6930


In [6]:
with open(path_embedding, 'rb') as file:
    embedding_list = pickle.load(file)

In [7]:
try:
  with open(os.path.join(path_save, 'list_image.pkl'), 'rb') as file:
    list_of_image = pickle.load(file)
  
except:
  list_of_image = []

try:
  with open(os.path.join(path_save, 'analyzed.pkl'), 'rb') as file:
    list_analyzed = pickle.load(file)
  
except:
  list_analyzed = []

print(len(list_analyzed), len(list_of_image))

4406 81


In [8]:
for i, image_path in tqdm(enumerate(original_photo_path)):
    gc.collect()

    if image_path in list_analyzed:
      continue
    
    try:
      image = face_recognition.load_image_file(image_path)

    except:
      list_analyzed.append(image_path)
      continue

    h, w, _ = image.shape
    
    max_size = max(h, w)
    scale = shape/max_size

    if scale > 1:
      new_h, new_w = int(h * scale), int(w * scale)

      image = cv2.resize(image, dsize=(new_w, new_h), interpolation=cv2.INTER_CUBIC)
    else:
      pass

    face_embedding = face_recognition.face_encodings(image, model = 'cnn')

    if len(face_embedding) == 0:
      list_analyzed.append(image_path)
      continue
      
    check_all = [np.mean(face_recognition.compare_faces(embedding_list, x)) for x in face_embedding]

    list_analyzed.append(image_path)

    if max(check_all) > 0.5:
      list_of_image.append(image_path)
      
      with open(os.path.join(path_save, 'list_image.pkl'), 'wb') as file:
        pickle.dump(list_of_image, file, protocol=pickle.HIGHEST_PROTOCOL)
      
      with open(os.path.join(path_save, 'analyzed.pkl'), 'wb') as file:
        pickle.dump(list_analyzed, file, protocol=pickle.HIGHEST_PROTOCOL)

    if i % 100 == 0:
      with open(os.path.join(path_save, 'list_image.pkl'), 'wb') as file:
        pickle.dump(list_of_image, file, protocol=pickle.HIGHEST_PROTOCOL)
      
      with open(os.path.join(path_save, 'analyzed.pkl'), 'wb') as file:
        pickle.dump(list_analyzed, file, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
with open(os.path.join(path_save, 'list_image.pkl'), 'wb') as file:
  pickle.dump(list_of_image, file, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(path_save, 'analyzed.pkl'), 'wb') as file:
  pickle.dump(list_analyzed, file, protocol=pickle.HIGHEST_PROTOCOL)